In [1]:
# Erasmus+ ICCT project (2018-1-SI01-KA203-047081)

# Toggle cell visibility

from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

# Hide the code completely

# from IPython.display import HTML
# tag = HTML('''<style>
# div.input {
#     display:none;
# }
# </style>''')
# display(tag)


## Szabályozás állapot visszacsatolással - Jelkövetés

Az adott rendszert figyelembe véve:

$$
\dot{x}=\underbrace{\begin{bmatrix}-3&4\\0&2\end{bmatrix}}_{A}x+\underbrace{\begin{bmatrix}0\\1\end{bmatrix}}_{B}u,
$$

a cél, hogy az első állapot kövesse a 6 rad/s ($\approx 1$ Hz) szinuszos referenciát lényegében nulla amplitúdó hibával.

Első lépésként hozzáadásra kerül egy integrátor (egy új állapot felvételével, mint az a [Szabályozás állapot visszacsatolással - Állandósult állapot](SS-31-Állapot_visszacsatolás_állndósult_állapot.ipynb) példában látható, megerősítve, hogy a kiegészített rendszer irányítható), hogy a zárt huroknak legyen egy átviteli függvénye, ami a referenciától $x_1$-ig tart és 0dB-ről indul. Az eredményül kapott rendszer:

$$
\dot{x}_a=\underbrace{\begin{bmatrix}-3&4&0\\0&2&0\\1&0&0\end{bmatrix}}_{A_a}x_a+\underbrace{\begin{bmatrix}0\\1\\0\end{bmatrix}}_{B_a}u+\underbrace{\begin{bmatrix}0\\0\\-1\end{bmatrix}}_{B_{\text{ref}}}x_{1r}\,.
$$

Hogy a teljesítmény a rendszernek megfelelő legyen, célszerű meghatározni az átviteli függvény alakját, ami biztosítja a kívánt viselkedést, azaz 0dB $\omega=0$-tól legalább $\omega=6$ rad/s-ig és 0 fázis ugyanezen az intervallumon. Ennek az alaknak tudatában a megoldás, figyelembe véve a pólusok hatását az előttük lévő frekvenciákra, egyszerű - elég a 65 rad/s-nél nagyobb frekvenciákra pólust áthelyezni.

A választott pólusk így $\lambda_{1,2,3}= 65$ rad/s, az erősítés mátrixa pedig $K_a = \begin{bmatrix}3024.75&194&68656.25\end{bmatrix}^T$. A szabályozott rendszer:

$$
\dot{x}_a=\underbrace{\begin{bmatrix}-3&4&0\\-3024.75&-192&-68656.25\\1&0&0\end{bmatrix}}_{A_a-B_aK_a}x_a+\underbrace{\begin{bmatrix}0\\1\\0\end{bmatrix}}_{B_a}v+\underbrace{\begin{bmatrix}0\\0\\-1\end{bmatrix}}_{B_{\text{ref}}}x_{1r}
$$

aminek szimulációja alább látható Bode diagram formájában $x_{1r}$ referenciától az $x_1$ állapotig.

### Hogyan működik a példa?
Próbálja meg elérni, hogy a fázis hibája a rendszernek is zérus legyen. Milyen messze kell helyezni a pólusokat?

In [2]:
%matplotlib inline
import control as control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

In [3]:
# Preparatory cell

A = numpy.matrix('-3 4 0; 0 2 0; 1 0 0')
B = numpy.matrix('0; 1; 0')
Br = numpy.matrix('0; 0; -1')
C = numpy.matrix('1 0 0')
X0 = numpy.matrix('0; 0; 0')
K = numpy.matrix([842.25,104,10718.75])

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Brw = matrixWidget(3,1)
Brw.setM(Br)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-65])) 
eig2c.setM(numpy.matrix([[-65],[0]]))
eig3c.setM(numpy.matrix([-65]))

In [4]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Vizsgálat',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Vizsgálat',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['K beállítása', 'Sajátértékek beállítása'],
    value= 'Sajátértékek beállítása',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues for the observer
selc = widgets.Dropdown(
    options= ['0 komplex sajátérték', '2 komplex sajátérték'],
    value= '0 komplex sajátérték',
    description='Sajátértékek:',
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzus', 'egységugrás', 'szinusz', 'négyszögjel'],
    value='szinusz',
    description='Referencia típusa:',
    disabled=False,
    style = {'description_width': 'initial'}
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=20.0,
    step=0.1,
    description='Referencia:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=1,
    min=0.01,
    max=4,
    step=0.01,
    description='Periódus: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [5]:
# Support functions

def eigen_choice(selc):
    if selc == '0 komplex sajátérték':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eigc = 0
    if selc == '2 komplex sajátérték':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eigc = 2
    return eigc

def method_choice(selm):
    if selm == 'K beállítása':
        method = 1
        selc.disabled = True
    if selm == 'Sajátértékek beállítása':
        method = 2
        selc.disabled = False
    return method

In [6]:
def main_callback(Aw, Bw, Brw, X0w, K, eig1c, eig2c, eig3c, u, period, selm, selc, selu, DW):
    A, B, Br = Aw, Bw, Brw 
    sols = numpy.linalg.eig(A)
    eigc = eigen_choice(selc)
    method = method_choice(selm)
    
    if method == 1:
        sol = numpy.linalg.eig(A-B*K)
    if method == 2:
        if eigc == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K) 
        if eigc == 2:
            K = control.acker(A, B, [eig1c[0,0], 
                                      numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                      numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
        sol = numpy.linalg.eig(A-B*K)
    print('A rendszer sajátértékei:',round(sols[0][0],4),',',round(sols[0][1],4),'és',round(sols[0][2],4))
    print('A szabályozott rendszer sajátértékei:',round(sol[0][0],4),',',round(sol[0][1],4),'és',round(sol[0][2],4))
    
    sys = sss(A-B*K,Br,C,0)
    T = numpy.linspace(0, 6, 1000)
      
    if selu == 'impulzus': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'egységugrás':
        U = [u for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'szinusz':
        U = u*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    if selu == 'négyszögjel':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys,T,U,X0w)
    
    fig = plt.figure(num='Bode diagram', figsize=(16,10))
    control.bode_plot(sys)
    fig.suptitle('Bode diagram', fontsize=16)
    
    plt.figure(num='Szimuláció', figsize=(16,4))
    plt.title('Első állapot válasza')
    plt.ylabel('$X_1$ vs ref')
    plt.plot(T,xout[0],T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.legend(['$x_1$','Referencia'])
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()

    
alltogether = widgets.VBox([widgets.HBox([selm, 
                                          selc, 
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Sajátértékek:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('Rendszermátrix Aa:',border=3),
                                          Aw,
                                          widgets.Label('Bemenet mátrix Ba:',border=3),
                                          Bw,
                                          widgets.Label('Referencia mátrix Br:',border=3),
                                          Brw])])
out = widgets.interactive_output(main_callback, {'Aw':Aw, 'Bw':Bw, 'Brw':Brw, 'X0w':X0w, 'K':Kw, 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 
                                                 'u':u, 'period':period, 'selm':selm, 'selc':selc, 'selu':selu, 'DW':DW})
out.layout.height = '1050px'
display(out, alltogether)

Output(layout=Layout(height='1050px'))